In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

from sklearn import linear_model

## (Non-)Linear function approximation
Our goal is to test how to do function approximation. 
We consider the following functions, all with the domain $X \equiv \mathbb{R}^{n}$ with $n=3$:

Linear:
$$
f_1 \mapsto \sum\limits_{i=1}^{3} x_i.
$$

Quadratic
$$
f_2 \mapsto \sum\limits_{i=1}^3 x_i^2.
$$

Simple periodic
$$
f_3 \mapsto \sin(x_1) + \cos(x_2) + \sin(x_3)\cos(x_3)
$$

Simple step-wise: (all terms must have absolute value less than 5)
$$
f_4 \mapsto \prod\limits_{i=1}^3 \mathbf{1}[\vert x_i \vert \leq 5].
$$

Another simple step-wise:
$$
f_5 \mapsto \Big \lfloor \ln\Big(1 + \max\big( 0, \sum\limits_{i=1}^j \prod\limits_{j \ne i} x_j\big) \Big ) \Big \rfloor.
$$

In [ ]:
n = 3

def f1(x):
    return np.sum(x)

def f2(x):
    return np.sum(np.square(x))

def f3(x):
    return np.sin(x[0]) + np.cos(x[1]) + np.sin(x[2]) * np.cos(x[2])

def f4(x):
    return np.prod((np.abs(x) <= 5).astype('int'))
    
def f5(x):
    return np.floor(np.log(1+max(0, np.sum([np.prod(np.append(x[:i],x[i+1:])) for i in range(len(x))]))))

Here, we plot on the 1D projection:

$$
\alpha \mapsto f_k(\alpha \mathbf{1}_{n}),~~ k \in \{1,2,3,4,5\}.
$$

In [ ]:
xs = np.linspace(-10, 10, 1000)
ones = np.ones(n)
ax = plt.subplot()
ax.plot(xs, [f5(x*ones) for x in xs])
ax.set(
    title="Function values by affine transformation", 
    xlabel=r"$\alpha$", 
    ylabel=r"$f(\alpha \mathbf{1}_n)$"
)

## Monomial Fitting
First we will fit by polynomials (up to degree $d$)

$$\begin{aligned}
    v_{d\cdot i+j}(x) &= x_i^j, ~~ i=1,\ldots,n; j=1,\ldots,d.
\end{aligned}$$

Then we want to learn a weights vector $\omega \in \mathbb{R}^{d \cdot n}$ such that:
$$
    f_k(x) \approx v(x)^T\omega.
$$

We will evaluate over the domain [-50,50].

In [ ]:
# create training points and test points
def get_Ab(f, d, seed=None):
    n_train = 4*d*n
    n_test  = d*n
    
    rng = np.random.default_rng(seed)
    train_set = 10*rng.random(size=(n_train, n))-5
    test_set  = 10*rng.random(size=(n_test, n))-5
    
    A_train = np.zeros((n_train, d*n), dtype=float)
    A_test  = np.zeros((n_test, d*n), dtype=float)
    b_train = np.zeros(n_train, dtype=float)
    b_test  = np.zeros(n_test, dtype=float)
    
    for i,train_pt in enumerate(train_set): 
        V = np.vander(train_pt, increasing=True, N=d)
        A_train[i] = np.reshape(V, newshape=(-1,))
        b_train[i] = f(train_pt)

    for i,test_pt in enumerate(test_set):
        V = np.vander(test_pt, increasing=True, N=d)
        A_test[i] = np.reshape(V, newshape=(-1,))
        b_test[i] = f(test_pt)

    return (A_train, b_train, A_test, b_test)

Fit the data! Our degrees are chosen by simple tuning.

In [ ]:
fs = [f"f{i}" for i in range(1,6)]
ds = [2,3,3,3,3]

for fn,d in zip(fs, ds):
    (A_train, y_train, A_test, y_test) = get_Ab(locals()[fn], d)
    reg = linear_model.LinearRegression()
    reg.fit(A_train,y_train)
    omega = reg.coef_
    
    y = A_test@omega
    print(f"Error for {fn} with degree d={d}: {la.norm(y-y_test, ord=2)}")

Let's try it again with regularization (i.e., ridge regression).

In [ ]:
fs = [f"f{i}" for i in range(1,6)]
ds = [2,3,3,3,3]

for fn,d in zip(fs, ds):
    (A_train, y_train, A_test, y_test) = get_Ab(locals()[fn], d)
    reg = linear_model.Ridge(alpha=.1)
    reg.fit(A_train,y_train)
    omega = reg.coef_
    
    y = A_test@omega
    print(f"Error for {fn} with degree d={d}: {la.norm(y-y_test, ord=2)}")

## Fourier basis

Recall $x \in \mathbb{R}^n$. We define the basis
$$
v_i(x) = \cos(\pi \cdot x^Tc^{(i)}).
$$
where $c^{(i)}_j \in \{0,\ldots,n\}$. Noticing there are $(n+1)^n$ possible $c^{(i)}$ since the vector can only take on discrete values, although we do not need to use all of them. 

## Tiling Coding (Course Coding)

Tiling coding is a type of course coding, where binary features are activitated (either 0 or 1) depending on whether the point is within a certain tile.

Mathematically, for every $i$, there exists a unique $x_i \in X$ such that
$$\begin{aligned} v_i(x) = \begin{cases} 1 & : x \in \mathrm{tile}(x_i) \\ 0 & : \text{o.w.} \end{cases} \end{aligned}.$$
$$
Here, each $v_i$ decomposes the domain $X$ into (equal length) tiles, or hypercubes. We assume the decomposition of $X$ for $v_i$ and $v_j$ shifted by a constant that is a fraction of the tile width along all dimension (to prevent overlap in any dimension).

Such a feature mapping can be useful where there are geographically based features. For example, if to capture a function that gets larger as one gets closer to an origin, we define features that take on values of 1 only close to the center.

**I will not code this since the next feature is a smoothed/continuous version of tiling coding**.

## Brief Tour: What is Reproducing Kernel Hilbert Space?

Recall that we want to approximate a function $f : X \mapsto \mathbb{R}$ via
$$
f(x) \approx v(x)^T\omega
$$
for some $\omega \in \mathbb{R}^d$. For example, in linear SVM, we let $v = I$ and define the classifier as
$$
\mathrm{sign}(\omega^Tx - b).
$$
However, many data sets may not be linearly separable (i.e., separated by a hyperplane), so we need a nonlinear seperation. We want to lift $x \mapsto \varphi(x)$, where $\varphi : X \mapsto H \subseteq \mathbb{R}^d$ for some $d > n$, where $H$ is a Hilbert space. Then we can compute instead
$$
\omega^Tx \to \langle \omega, \varphi(x) \rangle_H.
$$
Here, we abused notation and assume $\omega$ is defined according the respective space.
Often times, the larger the lifted space $d$ is (possibly infinite dimensional), the better approximation power we get.
So while this lifting procedure is appealing, it can be computationally intractble when $d$ is large.

The idea of Reproducing Kernel Hilbert Space (RKHS) is to use a kernel $k: H \times H \to \mathbb{R}$ and the *kernel trick* to avoid the need to explicitly compute $\varphi$. The kernel trick is a consequence of *Mercer's Theorem*, which says under suitable conditions for a lifting $\varphi$, there exists a kernel $k$ such that 
$$
k(x,x') = \langle \varphi(x), \varphi(x') \rangle_H, ~~ \forall x,x' \in X.
$$
This is a sufficient result, so it holds both ways. The key insight here is if we can find $k$, we do not need to form $\varphi$, which can be very high-dimensional.

Let us now apply this kernel trick to function approximation. Suppose we already have some data $\{(x_i,y_i=f(x_i))\}$, and we want to approximate $f$. Let us we write our weights in Hilbert space (i.e., $y_i \cdot \varphi(x_i)$ is the basis -- **WHY??**)
$$
\omega = \sum\limits_{i} \alpha_i y_i \varphi(x_i).
$$
Using the kernel trick,
$$
\langle \omega, \varphi(x) \rangle_H  = \sum\limits_i \alpha_i y_i \underbrace{\langle \varphi(x_i), \varphi(x) \rangle_H}_{k(x_i,x)}.
$$

## Radial Basis Function

We define the basis
$$
v_i(x) = k(x,c^{(i)}) := \mathrm{exp}\Big( \frac{\|x-c^{(i)}\|^2}{2\sigma_i^2} \Big).
$$

Notice that $v_i(x) \in (0,1]$, hence this is a relaxed version of a binary feature. 

## Random Kitchen Sinks